In [7]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib ipympl
# %matplotlib

In [8]:
def trajGen(orientations,start_point=(513,513),init_ori=0,step=1/165.):
    '''
    get_trajectories_from_orientations
    
    orientations : array to generate trajectory from
    start point, tuple of xy start cordinate
    step : size of stem
    
    '''
    x = start_point[0]
    y = start_point[1]
    xl=[x]
    yl=[y]
    os=orientations[0]
    for ori in orientations:
        x = x - step*np.sin(np.pi*(-os+init_ori+ori)/180)
        y = y + step*np.cos(np.pi*(-os+init_ori+ori)/180)
        xl.append(x)
        yl.append(y)
    return(xl,yl)

In [23]:
def transition_matrix(transitions):
    n = 1+ max(transitions) #number of states

    M = [[0]*n for _ in range(n)]

    for (i,j) in zip(transitions,transitions[1:]):
        M[i][j] += 1

    #now convert to probabilities:
    for row in M:
        s = sum(row)
        if s > 0:
            row[:] = [f/s for f in row]
    return M

In [3]:
dfl=pd.read_hdf("all_wind_clean.h5")
files = dfl.timeStamp.unique()
df=dfl[(dfl.timeStamp ==files[2])]


In [61]:
win=40
dff=df[(df.trajectory__runNum==2) & (df.trajectory__case==2) & (df.trajectory__speed>0)]
wbad=dff.trajectory__wbad
wbadr=wbad.rolling(win).mean().dropna()

# ori=dff.trajectory__pOri_x

In [78]:
ws,wsb=pd.qcut(wbadr,3,retbins=True,labels=False)
binC=pd.DataFrame(wsb).rolling(2).mean().dropna().values.T[0]
wsd=pd.DataFrame(ws)
wsd['wsim']=binC[wsd['trajectory__wbad']]
wsim=wsd['wsim']

In [82]:
s=''.join(wsd['trajectory__wbad'].astype(str).values)
s2=np.array(list(map(''.join, zip(*[iter(s)]*3)))).astype(int)


In [81]:
np.array(transition_matrix(s2))

MemoryError: 

In [83]:
plt.figure()
plt.imshow(transition_matrix(s2))
# plt.imshow(transition_matrix(wsd['trajectory__wbad'].values))
# 
plt.colorbar()

/home/lsr/anaconda3/lib/python3.6/site-packages/matplotlib/pyplot.py:537: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

In [31]:
wbadrc=wbadr.cumsum()
wbadc=wbad.cumsum()[win-1:]
wsimc=wsim.cumsum()
wsimc8=-8*wsimc

x = np.arange(len(wbadc))/165.
xc= np.arange(len(wsimc))/165.


plt.figure()#smooth vs sim wbad
plt.plot(np.arange(len(wbadr))/165.,wbadr.values,label='actual wbad')#smooth wbad
plt.plot(np.arange(len(wsim))/165.,wsim.values,label='binned wbad')#sim wbad
plt.xlabel('Time (s)')
plt.ylabel('WBAD (deg)')
plt.legend()

plt.figure() #actual vs smooth vs sim heading
plt.plot(x,-8*wbadc,label='actual heading')#actual heading
plt.scatter(x,-8*wbadrc,s=5,label='smooth heading')#smooth heading
plt.plot(xc,-8*wsimc,label='binned heading')#simulated heading
plt.xlabel('Time (s)')
plt.ylabel('Heading (deg)')
plt.legend()


rotOff=-10
plt.figure() #actual vs sim trajectory
plt.plot(dff.trajectory__pPos_x,dff.trajectory__pPos_y,c='b',label='actual traj')#actual traj
plt.plot(trajGen(wsimc8,init_ori=rotOff)[0],trajGen(wsimc8,init_ori=rotOff)[1],c='r',label='binned traj')#sim traj
plt.xlabel('X (m)')
plt.ylabel('Y (m)')
plt.legend()

plt.axes().set_aspect('equal','datalim')

FigureCanvasNbAgg()

FigureCanvasNbAgg()

FigureCanvasNbAgg()

/home/lsr/anaconda3/lib/python3.6/site-packages/matplotlib/cbook/deprecation.py:107: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)


In [ ]:
plt.figure()
wbadr=wbad.rolling(win).mean().dropna()
wbadc=wbad.cumsum()[win-1:]
wsimc=wsim.cumsum()

# wbad.cumsum().plot()

wbadrc=wbadr.cumsum()
x = np.arange(len(wbadc))/165.
xc= np.arange(len(wsimc))/165.
y = wbadr
# col = np.where(y<-thresh,'r',np.where(y>thresh,'b','g'))

col = np.where(y<-thresh2,'k',np.where(y<-thresh1,'b',np.where(y<thresh1,'g',np.where(y<thresh2,'r','m'))))

plt.scatter(x,-8*wbadrc,color=col)
plt.plot(x,-8*wbadc)
plt.plot(xc,-8*wsimc)

In [ ]:
turnVel1 = 0.0404106
turnVel2 = 0.1
turnVel3 = 0.18

thresh1=0.01
thresh2=0.05

left1= -turnVel1
left2= -turnVel2
right1=turnVel1
right2=turnVel2
straight = 0


wbadr=wbad.rolling(win).mean().dropna()
wbadc=wbad.cumsum()[win-1:]
x = np.arange(len(wbadc))/165.
y = wbadr

wsim = np.where(y<-thresh2,left2,np.where(y<-thresh1,left1,np.where(y<thresh1,straight,np.where(y<thresh2,right1,right2))))
wsimc=wsim.cumsum()
xc= np.arange(len(wsimc))/165.

wbadrc=wbadr.cumsum()


simx,simy=trajGen(wsimc)

plt.figure()
plt.axhline(thresh1,c='r')
plt.axhline(-thresh1,c='b')
plt.axhline(straight,c='g')
plt.axhline(thresh2,c='m')
plt.axhline(-thresh2,c='k')

plt.plot(wbadr.values)
plt.plot(wsim)


In [ ]:
[wbadr[np.where(y>thresh)[0]].mean(),wbadr[np.where(y<-thresh)[0]].mean()]

In [ ]:
plt.figure()
plt.plot(dff.trajectory__pPos_x,dff.trajectory__pPos_y)
plt.axes().set_aspect('equal','datalim')

In [ ]:
thresh=0.05
win=5
wbadr=wbad.rolling(win).mean().dropna()
x = np.arange(len(wbadr))
y = wbadr

col = np.where(y<-thresh,'r',np.where(y>thresh,'b','g'))
plt.figure()

plt.plot(wbadr.values,alpha=0.8)
plt.scatter(x, y, c=col, s=10, linewidth=0)
plt.xlim(500,800)
plt.show()

In [ ]:
plt.figure()
plt.plot(ori)

In [ ]:
plt.figure()
plt.plot(wbad,color=['k']*2397)

In [ ]:
plt.figure()
# plt.plot(wbad,'k')
#plt.plot(wbadr)
proxy = np.zeros(wbadr.values.shape[0])
proxy[(wbadr>0.1) ] = wbadr[(wbadr>0.1) ]
plt.plot(wbadr.values)
plt.plot(proxy)



In [ ]:
plt.figure()

plt.hist(wbadr.values,10)

In [ ]:
from bokeh.io import output_file, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure

output_file("panning.html")

x = list(range(11))
y0 = x
y1 = [10-xx for xx in x]
y2 = [abs(xx-5) for xx in x]

# create a new plot
s1 = figure(plot_width=250, plot_height=250, title=None)
s1.circle(x, y0, size=10, color="navy", alpha=0.5)

# create a new plot and share both ranges
s2 = figure(plot_width=250, plot_height=250, x_range=s1.x_range, y_range=s1.y_range, title=None)
s2.triangle(x, y1, size=10, color="firebrick", alpha=0.5)

# create a new plot and share only one range
s3 = figure(plot_width=250, plot_height=250, x_range=s1.x_range, title=None)
s3.square(x, y2, size=10, color="olive", alpha=0.5)

p = gridplot([[s1, s2, s3]], toolbar_location=None)